# IMPORTS

In [1]:
import os
import json
import itertools

from osgeo import gdal
from datetime import datetime, timedelta
from collections import Counter

# 0 => clear land pixel
# 1 => clear water pixel
# 2 => cloud shadow
# 3 => snow
# 4 => cloud

In [2]:
directory = f"/Volumes/X/Data/fusion-s1-s2/s2/fmask/"  # replace with the path to your directory
fmask_paths = [
    f"/Volumes/X/Data/fusion-s1-s2/s2/fmask/" +
    folder_name +
    "/" +
    folder_name +
    "_fmask.tif"
    for folder_name in os.listdir(directory)
]

In [3]:
MAP_CLOUD = {
    0: "clear_land",
    1: "cloud_water", 
    2: "cloud_shadow",
    3: "snow",
    4: "cloud",
    255: "no_observation"
}

In [4]:
def fmask_double(input_path, output_path):
    if os.path.isfile(input_path):
        src_ds = gdal.Open(input_path)
    else:
        src_ds = gdal.Open("/".join(input_path.split("/")[:-1])+"/fmask.tif")

    # Calculate the new dimensions
    new_width = src_ds.RasterXSize * 2
    new_height = src_ds.RasterYSize * 2

    # Set up the warp options
    warp_options = gdal.WarpOptions(format="VRT",
                                    width=new_width,
                                    height=new_height,
                                    resampleAlg=gdal.GRA_Bilinear)
    
    gdal.Warp(destNameOrDestDS=output_path, srcDSOrSrcDSTab=src_ds, options=warp_options)

In [ ]:
for fmask_path in fmask_paths:
    file_name = "_".join(fmask_path.split("/")[-1].split("_")[:-1])
    output_path = f"data/fmask_cropped/{file_name}/{file_name}.vrt"
    if not os.path.exists(f"data/fmask_cropped/{file_name}/"):
                os.makedirs(f"data/fmask_cropped/{file_name}/")
    fmask_double(fmask_path, output_path)

In [60]:
CROP_SIZE = 256

In [ ]:
for fmask in os.listdir("data/fmask_cropped/"):
    print(fmask)
    image_path = f"data/fmask_cropped/{fmask}/{fmask}.vrt"
    image = gdal.Open(image_path)
    print(f"Processing {image_path}")

    width = image.RasterXSize
    height = image.RasterYSize

    gt = image.GetGeoTransform()

    min_x = int(gt[0])
    min_y = int(gt[3]) - CROP_SIZE * 10
    max_x = int(gt[0] + width*gt[1])
    max_y = int(gt[3] + height*gt[5]) - CROP_SIZE * 10

    x_length = range(min_x, max_x + CROP_SIZE + 1, CROP_SIZE*10)
    y_length = range(min_y, max_y - CROP_SIZE + 1, -CROP_SIZE*10)

    ALL_XY_COORDINATES = [(x, y) for y in y_length for x in x_length]

    if not os.path.isdir(f"data/fmask_cropped/{fmask}/cropped/"):
        os.makedirs(f"data/fmask_cropped/{fmask}/cropped/")
    for idx, (x, y) in enumerate(ALL_XY_COORDINATES):
        vrt_options = gdal.BuildVRTOptions(resolution="highest", outputBounds=(x, y, x + CROP_SIZE * 10, y + CROP_SIZE * 10))
        gdal.BuildVRT(f"data/fmask_cropped/{fmask}/cropped/{fmask}_{idx}_{x}_{y}_{CROP_SIZE}.vrt", [image_path], options=vrt_options)

In [ ]:
json_data = {}

FMASK_DIRECTORY = f"data/fmask_cropped/"

for fmask_path in os.listdir(FMASK_DIRECTORY):
    print(f"Processing {fmask_path}")
    path_to_crops = f"{FMASK_DIRECTORY}{fmask_path}/cropped/"
    for im_path in os.listdir(path_to_crops):
        x, y = int(im_path.split("_")[-3]), int(im_path.split("_")[-2])
        gdal_data = gdal.Open(f"{FMASK_DIRECTORY}{fmask_path}/cropped/{im_path}")
        data = gdal_data.ReadAsArray()
        
        shape = data.size
        
        count = dict(Counter(itertools.chain(*data)))
        map = {MAP_CLOUD[k]: round((v/shape)*100, 2) for k, v in count.items()}
        
        json_data[im_path.split("/")[-1]] = dict(sorted(map.items(), key=lambda x: -x[1]))

with open("data/fmask_cropped_stats.json", "w") as f:
    json.dump(json_data, f)

# Filter Unwanted data, and separate cloudy and cloud free, closest date

In [5]:
with open("data/fmask_cropped_stats.json", "r") as f:
    fmask_data = json.load(f)

Remove the mark of no observation (orbits)

In [6]:
fmask_data_filtered_no_observation = {}

for k, v in fmask_data.items():
    if "no_observation" not in v or v["no_observation"] < 1:
        fmask_data_filtered_no_observation[k] = v


Remove the water

In [7]:
fmask_filtered_obs_water = {}

for k, v in fmask_data_filtered_no_observation.items():
    if "cloud_water" not in v or ("cloud_water" in v and v["cloud_water"] < 1):
        fmask_filtered_obs_water[k] = v

Filter cloudy images ...

In [8]:
cloudy_fmask_data = {}

for k, v in fmask_filtered_obs_water.items():
    if "cloud" in v and v["cloud"] > 10:
        cloudy_fmask_data[k] = v

Filter Cloud free images

In [9]:
cloud_free_fmask_data = {}

for k, v in fmask_filtered_obs_water.items():
    if "cloud" not in v or v["cloud"] < 1:
        cloud_free_fmask_data[k] = v

# Create all the candidates data

Utility function to find the cloud free image that is the closest.

In [10]:
def find_closest_date_and_coordinates(target_date, target_x, target_y):
    target_date = datetime.strptime(target_date, "%Y%m%d")

    # Initialize the minimum difference and the result
    min_diff = None
    result = None

    for k, v in cloud_free_fmask_data.items():
        date, x, y = k.split("_")[2], k.split("_")[4], k.split("_")[5]
        if x == target_x and y == target_y:
            # Convert item date string to datetime object
            item_date = datetime.strptime(date, "%Y%m%d")

            # Calculate the time difference
            diff = abs(target_date - item_date)

            # Update the result if the difference is smaller than the current minimum
            if min_diff is None or diff < min_diff:
                min_diff = diff
                result = k
    if result:
        return result, min_diff

In [11]:
CANDIDATES_FMASK = {}
idx = 0
for k, v in cloudy_fmask_data.items():
    date, x, y = k.split("_")[2], k.split("_")[4], k.split("_")[5]
    if s:= find_closest_date_and_coordinates(date, x, y):
        if s[1] <= timedelta(days=5):
            CANDIDATES_FMASK[idx] = {
                "cloudy": k,
                "cloud_free": s[0] 
            }
            idx += 1


In [12]:
len(CANDIDATES_FMASK)

118251

In [13]:
with open("data/candidates_filtered_water_MAXED.json", "w") as f:
    json.dump(CANDIDATES_FMASK, f)